<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [ ]:
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys

from datetime import timedelta, datetime, date

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor


import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import f_regression, mutual_info_regression

import json
import matplotlib.pyplot as plt
import seaborn as sns


warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None) #отображение всех столбцов датасета

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))
#  Функция по сравнению дата-сетов (найдена на каггле)
def check_df_1vs2(d_df1,d_df2):
    
    list_of_names1 = list(d_df1.columns)
    temp_dict = {}
    temp_dict['feature_train'] = list_of_names1
    temp_dict['type_train'] = d_df1.dtypes
    temp_dict['sample_train'] = d_df1.loc[5].values
    temp_dict['# unique_train'] = d_df1.nunique().values
    temp_df1 = pd.DataFrame.from_dict(temp_dict)
    
    
    list_of_names2 = list(d_df2.columns)
    temp_dict2 = {}
    temp_dict2['feature_test'] = list_of_names2
    temp_dict2['type_test'] = d_df2.dtypes
    temp_dict2['sample_test'] = d_df2.loc[5].values
    temp_dict2['# unique_test'] = d_df2.nunique().values
    temp_df2 = pd.DataFrame.from_dict(temp_dict2)
    
    temp_insert = pd.DataFrame(columns=['< - >'])
    
    temp_df = pd.concat([temp_df1,temp_insert, temp_df2], axis=1, sort=False)
    temp_df.reset_index(inplace = True)
    del temp_df['index']
    temp_df['< - >'] = '| - |'
#    display(temp_df)

    temp_dict3 = {}
    temp_df3= pd.DataFrame(temp_df)
    temp_list  = []
    temp_list2  = []
    temp_list3  = []
    temp_list4  = []
    temp_list5  = []

    for i in range(len(temp_df)):
        if str(temp_df3['type_train'][i]) != str(temp_df3['type_test'][i]):
            temp_list.append(temp_df3['feature_train'][i])
            temp_list2.append(temp_df3['feature_test'][i])
            temp_list3.append(str(temp_df3['type_train'][i]) + ' != ' + str(temp_df3['type_test'][i]))
            temp_list4.append(i)
        if temp_df3['# unique_test'][i]>0 and temp_df3['# unique_train'][i]/temp_df3['# unique_test'][i] > 2:
            temp_list5.append(i)
            
    temp_dict3['index']= temp_list4
    temp_dict3['feature_train']= temp_list
    temp_dict3['не совпадают типы'] = temp_list3
    temp_dict3['feature_test']= temp_list2

    temp_df4 = pd.DataFrame.from_dict(temp_dict3)
    temp_df4.set_index('index',inplace=True)

    print(f'Резюме:\n 1. Не совпали типы в:= {len(temp_df4)} столбцах\n')
    print(f'2. Уникальные значения различаются в:= {len(temp_list5)} столбцах {temp_list5}')
    display(temp_df4)
    
def print_report(start, y_test, y_pred):
    print('\nВремя выполнения - ', datetime.now() - start)
    print(f"Точность по метрике MAPE:{(mape(y_test, y_pred))*100:0.2f}%")

# Визуализация числовых данных
def graph_num(col, df, size=6):
    fig, (g1, g2) = plt.subplots(1, 2, figsize=(2*size, size))
    fig.suptitle('Histogram and boxplot for {0}'.format(col), fontsize=20)
    g1.hist(df[col], bins=20, histtype='bar', align='mid',
            rwidth=0.8, color='green')  # гистограмма
    g2.boxplot(df[col], vert=False)  # выбросы
    plt.figtext(0.5, 0, col, fontsize=16)
    plt.show

# Визуализация корреляции числовых признаков между собой
def jointplot_f(col_num, df):
    pairs = list(itertools.combinations(col_num, 2))
    for pair in pairs:
        sns.jointplot(x=pair[0], y=pair[1], data=df)
    return

# Статистические данные по выбросам
def statistic(col, df):
    median = df[col].median()
    IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
    perc25 = df[col].quantile(0.25)
    perc75 = df[col].quantile(0.75)
    l = perc25 - 1.5*IQR
    r = perc75 + 1.5*IQR
    print("Для {0} IQR: {1}, ".format(col, IQR),
          "Границы выбросов: [{0}, {1}].".format(l, r))
    print('Всего {} выбросов'.format(
        df[df[col] > r][col].count()+df[df[col] < l][col].count()))

# Setup

In [ ]:
VERSION    = 16
DIR_TRAIN1  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TRAIN  = '../input/auto-ru-26-05-2021-new/' # подключил к ноутбуку внешний датасет
DIR_TRAIN_CUR  = '../input/exchange-rate/'

DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [ ]:
!ls '../input'

In [ ]:
train = pd.read_csv(DIR_TRAIN+'auto_ru_26_05_2021.csv') # датасет для обучения модели
train_old = pd.read_csv(DIR_TRAIN1+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
exchange_rate = pd.read_csv(DIR_TRAIN_CUR+'exchange_currency.csv') # курс доллара

test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
exchange_rate

In [ ]:
train.info()

In [ ]:
'''train['price'] = train['price'].str.replace('\xa0', '')
train['price'] = train['price'].str.replace('₽', '')
train['price'] = train['price'].str.replace('от ', '').astype(int)
train['price'].isna().sum()'''

In [ ]:
train.head(5)

In [ ]:
test.info()

In [ ]:
test.productionDate.unique()

In [ ]:
test.isna().sum()

In [ ]:
test.columns

In [ ]:
len(test)

In [ ]:
check_df_1vs2(test,train)

## Анализ данных

In [ ]:
# По сравнению с тестовой выборкой в тренировочную не попали 7 признаков:
# 'image', 'priceCurrency', 'sell_id' - малозначимые признаки

# 'description', 'name', 'vehicleConfiguration' - описательная информация различной структуры,
# которая частично уже содержится в других признаках (останется время, попробую разобрать)

# 'modelDate' - потенциально интересный признак, но информации, с какого года выпускается модель,
# на сайте не нашел


# определим первичный список признаков
init_columns = ['bodyType', 'brand','color', 'car_url', 'complectation_dict', 'engineDisplacement','enginePower',
           'equipment_dict', 'fuelType', 'mileage', 'model_info', 'model_name', 'numberOfDoors',
           'productionDate', 'super_gen', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение',
           'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня']
cat_columns = []
ord_columns = []
num_columns = []
del_columns = []


df_train = train[init_columns]
df_test = test[init_columns]

In [ ]:
df_train['price'] = train['price']

In [ ]:
df_test.bodyType.unique(), df_train.bodyType.unique()
# структура и формат данных похож. Будем рассматривать как категориальный признак


In [ ]:
cat_columns.append('bodyType')
cat_columns

In [ ]:
df_test.brand.unique(), df_train.brand.unique() # не совпадает регистр.

In [ ]:
# Приведем к виду теста и будем рассматривать как категориальный признак
df_train.brand = df_train.brand.str.upper()
# Проверим
cat_columns.append('brand')

In [ ]:
test.color.unique(), train.color.unique()
#структура и формат данных похож. Будем рассматривать как категории

In [ ]:
cat_columns.append('color')

In [ ]:
df_test.complectation_dict.unique()[:2], df_train.complectation_dict.unique()[:2]
# структура и формат данных похож. Потом попробуем преобразовать и извлечь что-то 

In [ ]:
df_test.engineDisplacement.unique()[:5], df_train.engineDisplacement.unique()[:5]

In [ ]:
# Можно попробовать сделать и ординальный и числовой признак
# Сдесь занесем в ординальные, позже сделаем числовой
ord_columns.append('engineDisplacement')

In [ ]:
df_test.enginePower.unique()[:5], df_train.enginePower.unique()[:5]

In [ ]:
# Можно попробовать сделать и ординальные и числовой признак
# Сдесь занесем в ординальные, позже сделаем числовой
ord_columns.append('enginePower')

In [ ]:
df_test.equipment_dict.unique()[:2], df_train.equipment_dict.unique()[:2]
# Структура понятна, можно разобрать и попробовать label encoding. Сам признак скорее всего придется удалить.
# df_test.equipment_dict.apply(lambda x: json.loads(x) if type(x) == str else x)

In [ ]:
del_columns.append('equipment_dict')

In [ ]:
df_test.fuelType.unique(), df_train.fuelType.unique()

In [ ]:
# Приведем к виду тестовой выборки и будем рассматривать как категориальный признак
df_train.fuelType = df_train.fuelType.str.lower()
cat_columns.append('fuelType')

In [ ]:
df_test.mileage.unique(), df_train.mileage.unique()

In [ ]:
num_columns.append('mileage')

In [ ]:
df_test.model_info.unique()[:5], df_train.model_info.unique()[:5]

In [ ]:
# Похоже отсюда можно взять только имя модели. Но оно уже есть.
# Скорее всего удалим.
del_columns.append('model_info')

In [ ]:
df_test.model_name.unique()[:5], df_train.model_name.unique()[:5]
# Структура и тип данных совпадают. Будем рассматривать как категориальный признак.

In [ ]:
cat_columns.append('model_name')

In [ ]:
df_test.numberOfDoors.unique(), df_train.numberOfDoors.unique()
#структура и формат данных совпадают. Будем рассматривать категориальный признак

In [ ]:
cat_columns.append('numberOfDoors')

In [ ]:
df_test.productionDate.unique(), df_train.productionDate.unique()
#структура и формат данных совпадают. Будем рассматривать как категориальный признак

In [ ]:
num_columns.append('productionDate')

In [ ]:
df_test.super_gen.unique()[:3], df_train.super_gen.unique()[:3]
# структура и формат данных похожи. Данные вразнобой. В свежей выборке с auto.ru данные не совпадают.
# Однако, для тренировочной выборки отсюда можно взять недостающий год модели. Интересен еще price_segment,
# но его нет в тестовой выборке. А acceleration, clearance_min и т.д. не попались при парсинге. Может вернемся.
# Сам признак, скорее всего придется удалить

In [ ]:
df_train['super_gen'] = df_train['super_gen'].apply(lambda x: json.loads(x) if type(x) == str else x)
df_train['modelDate'] = df_train['super_gen'].apply(lambda x: x['year_from'] if type(x) == dict else x)
del_columns.append('super_gen')

In [ ]:
df_test['modelDate'] = test['modelDate']

In [ ]:
df_test.modelDate.unique(), df_train.modelDate.unique()

In [ ]:
num_columns.append('modelDate')

In [ ]:
df_test.vehicleTransmission.unique(), df_train.vehicleTransmission.unique()

In [ ]:
#структура и формат данных совпадают. Будем рассматривать как категориальный признак
cat_columns.append('vehicleTransmission')

In [ ]:
df_test.vendor.unique(), df_train.vendor.unique()

In [ ]:
#структура и формат данных совпадают. Будем рассматривать как категориальный признак
cat_columns.append('vendor')

In [ ]:
df_test.Владельцы.unique(), df_train.Владельцы.unique()
#структура и формат данных похож, но надо подчистить пробелы. Будем рассматривать как категориальный признак
df_test.Владельцы = df_test.Владельцы.str.replace('\xa0', ' ')
df_train.Владельцы = df_train.Владельцы.str.replace('\xa0', ' ')

In [ ]:
df_test.Владельцы.unique(), df_train.Владельцы.unique()

In [ ]:
#структура и формат данных теперь совпадают. Будем рассматривать как ординальный признак
ord_columns.append('Владельцы')

In [ ]:
df_test.Владение.unique()[:5], df_train.Владение.unique()[:5]
# структура и формат данных похож. Можно попробовать перевести в численный признак.
# Но в тестовом наборе очень много пропусков. Если заполнить эффективно не удастся, попробуем удалить.
# Результат сравним и примем решение.

In [ ]:
df_test.ПТС.unique(), df_train.ПТС.unique()

In [ ]:
#структура и формат данных совпадает. Будем рассматривать как категориальный признак
cat_columns.append('ПТС')

In [ ]:
df_test.Привод.unique(), df_train.Привод.unique()

In [ ]:
#структура и формат данных совпадает. Будем рассматривать как категориальный признак
cat_columns.append('Привод')

In [ ]:
df_test.Руль.unique(), df_train.Руль.unique()

In [ ]:
#структура и формат данных совпадают. Будем рассматривать как категориальный признак
cat_columns.append('Руль')

In [ ]:
df_test.Состояние.unique(), df_train.Состояние.unique()

In [ ]:
#структура и формат данных совпадает. Будем рассматривать как категориальный признак
cat_columns.append('Состояние')

In [ ]:
df_test.Таможня.unique(), df_train.Таможня.unique()

In [ ]:
#структура и формат данных совпадает. Будем рассматривать как категориальный признак
cat_columns.append('Таможня')

In [ ]:
num_columns

In [ ]:
#train.dropna(subset=['productionDate','mileage'], inplace=True)
#train.dropna(subset=['price'], inplace=True)

In [ ]:
df_train.head(3)

# Соединяем тестовую и тренировочную выборки

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

# Поправочный коэффициент

In [ ]:
# Если попытаться сформулировать задачу с поправочным коэффициентом, она может выглядеть так.
# Рассчитать сколько бы стоила та или иная машина в ценах на момент создания некоей невидимой
# тестовой выборки (можно посмотреть по parsing_unixtime).
# Попробуем методом эталонов, то есть рассчитаем стоимость машин на (весну?) 2020 год и возьмем
# среднее отклонение.

In [ ]:
train_old['model_name'] = train_old['model']
train_price = train.append(train_old, sort=False).reset_index(drop=True)

In [ ]:
model_list = train_price['model_name'].unique()

In [ ]:
# Пробовал привязаться к старой выборке, чтобы определить коэффициенты изменения цены, но хороших результатов
# это не принесло. Видимо все сбивает сезонный фактор.
# Было бы понятнее, если была бы известна дата создания тестого набора на kaggle
# Определение ценовой поправки через курс доллара на данный момент выглядит предпочтительнее.
# Сделаем это по новому признаку rate.

'''price_benchmark = pd.DataFrame(columns=['year', 'model'])
years = [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021]
for year in years:
    for model in model_list:
        df1 = pd.DataFrame({'year': [year], 'model': [model]})
               
        df1['price'] = train_old[(train_old['model_name'] == model) &
                                   (train_old['productionDate'] == year)]['price'].mean()
        price_benchmark = price_benchmark.append(df1, sort=False).reset_index(drop=True)'''

In [ ]:
#price_benchmark[price_benchmark['year'] == 2020]['price'].isna().sum()

In [ ]:
# Отношение средневзвешенных курсов доллара

In [ ]:
exchange_rate = exchange_rate.rename(columns = {'year':'productionDate'})
data = data.merge(exchange_rate, how='left')
num_columns.append('rate')

a = data.rate.median()
data.rate = data.rate.fillna(a)

In [ ]:
data['rate_norm'] = data['rate'].apply(lambda x: (x*0.96714)**2)
num_columns.append('rate_norm')

In [ ]:
# Дубликаты
data.drop_duplicates(subset=['car_url'], keep='first', inplace=True, ignore_index=True)
del_columns.append('car_url')

In [ ]:
len(data.query('sample == 0'))

In [ ]:
data.head(3)

In [ ]:
len(test)

# Обработка признаков

In [ ]:
# Поскольку уникальных индексов в датасете нет, сделаем реиндексацию
data = data.reset_index()
data.rename({'index':'idx_df'}, axis = 1, inplace = True)

In [ ]:
data['complectation_dict'][0:10]

In [ ]:
# Даже при извлечении названия комплектации очень много пропусков, в том числе и в тестовом наборе.
# К тому же в записях очень много ошибок - лучше удалить
data['complectation_dict'] = data['complectation_dict'][0:1000].apply(
    lambda x: json.loads(x) if type(x) == str else x)
data['complectation_name'] = data['complectation_dict'].apply(
    lambda x: x.get('name') if type(x) == dict else x)
data['complectation_name'].isna().sum()

In [ ]:
cat_columns.append('complectation_name')
del_columns.append('complectation_dict')

In [ ]:
# engineDisplacementNum как числовой параметр (для тестирования)
import re

data['engineDisplacementNum'] = data['engineDisplacement'].apply(
    lambda x: x.split(' ')[0] if type(x) == str else x)
data['engineDisplacementNum'] = data['engineDisplacementNum'].replace('', None)
data['engineDisplacementNum'] = data['engineDisplacementNum'].apply(lambda x: float(x) if type(x) == str else x)
num_columns.append('engineDisplacementNum')

In [ ]:
# enginePowerNum как числовой параметр (для тестирования)
import re

data['enginePowerNum'] = data['enginePower'].apply(
    lambda x: x.split(' ')[0] if type(x) == str else x)
data['enginePowerNum'] = data['enginePowerNum'].replace('', None)
data['enginePowerNum'] = data['enginePowerNum'].apply(lambda x: int(x) if type(x) == str else x)
num_columns.append('enginePowerNum')

In [ ]:
data['equipment_dict'][9]

In [ ]:
data['equipment_dict'] = data['equipment_dict'].fillna('{"no":true}')

In [ ]:
#data['equipment_dict'] = 
data['equipment_dict_list'] = data['equipment_dict'].apply(
    lambda x: json.loads(x) if type(x) == str else x)

data['equipment_dict_list'] = data['equipment_dict_list'].apply(
    lambda x: list(x) if type(x) == dict else x)

#data['equipment_dict'] = data['equipment_dict'].apply(
#    lambda x: x.get('name') if type(x) == dict else x)

data['equipment_dict_list']

In [ ]:
data1 = data.explode('equipment_dict_list')

In [ ]:
data1.head(2)

In [ ]:
# Переведем 'Владение' в числовой признак 'OwnershipMonth'

import re
year_pattern = re.compile(r'([0123456789]*) \b(?:лет|год)')
month_pattern = re.compile(r'([0123456789]*) \bмесяц')

def ownership_year_month(text):
    if isinstance(text, float) or isinstance(text, int):
        return 0, text
    year, month = (0,0)
    m = year_pattern.search(text)
    if m:        
        year = int(m.group(1))
    m = month_pattern.search(text)
    if m:        
        month = int(m.group(1))
    return year, month


data['OwnershipMonth'] = data['Владение'].apply(lambda v: (
    lambda y, m: y*12+m)(*ownership_year_month(v)))

In [ ]:
data['Владение'][1010:1025], data['OwnershipMonth'][1010:1025]

In [ ]:
data['OwnershipMonth'] = np.log(2+data['OwnershipMonth'])
num_columns.append('OwnershipMonth')
del_columns.append('Владение')

In [ ]:
data.head(3)

In [ ]:
cat_columns

In [ ]:
ord_columns

In [ ]:
data.Владельцы.value_counts()

In [ ]:
data['Владельцы'] = data['Владельцы'].str.extract('(\d+)').astype(int)

In [ ]:
len(data.query('sample == 0'))

# Работа с пропусками и выбросами

In [ ]:
data.isna().sum()

In [ ]:
del_columns

In [ ]:
data[cat_columns].isna().sum()

In [ ]:
data.ПТС.value_counts()

In [ ]:
data.ПТС = data.ПТС.fillna('Оригинал')

In [ ]:
data[num_columns].isna().sum()

In [ ]:
OM_mean = data['OwnershipMonth'].mean()
data['OwnershipMonth'] = data['OwnershipMonth'].fillna(OM_mean)

In [ ]:
data['OwnershipMonth'].isna().sum()

In [ ]:
data[ord_columns].isna().sum()

In [ ]:
for col in(num_columns):
    graph_num(col, data)

In [ ]:
for col in num_columns:
    statistic(col, data)

In [ ]:
# Не смотря на то, что выбросы есть, и где-то значительные, в полном смысле выбросами это назвать нельзя.
# Вероятно поэтому, почти любые попытки их почистить приводят к ухудшению модели. 

In [ ]:
# Распределение почти нокмальное, со смещением. Попробуем прологарифмировать

#for i in num_columns:
#    data[i] = np.log(data[i]+1)

In [ ]:
# Нормализация данных
scaler = StandardScaler()
#scaler = MinMaxScaler()
for column in num_columns:
    data[column] = scaler.fit_transform(data[[column]])[:,0]

In [ ]:
# Посмотрим корреляцию числовых признаков
correlation = data[num_columns].corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

In [ ]:
data.rate

In [ ]:
# Значимость числовых признаков
imp_num = pd.Series(f_regression(data[data['sample'] == 1][num_columns],
                              data[data['sample'] == 1].price)[0], index = num_columns)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
del_columns.append('rate')

## Label Encoding

In [ ]:
# реализуем метод OneHotLabels через get_dummies для категориальных признаков

data = pd.get_dummies(data, prefix=cat_columns, columns=cat_columns)

In [ ]:
for colum in ['enginePower', 'engineDisplacement']:
    data[colum] = data[colum].astype('category').cat.codes

In [ ]:
#del_columns.append('enginePower')
#del_columns.append('engineDisplacement')

In [ ]:
# Создаем признаки с мнимыми переменными на основе параметра 'equipment_dict_list'
dummies = pd.get_dummies(data1['equipment_dict_list']).rename(columns=lambda x: str(x))
df1 = pd.concat([data1, dummies], axis=1)
df2 = pd.DataFrame()
#Составим общий список всех опций. Обеспечим уникальность через использование множеств
equipment_list = set()
for item in data1['equipment_dict_list']:
#    for i in item:
        equipment_list.add(item)
 #   equipment_list.add(i)

for i in equipment_list:
     df2[i] = df1.groupby(['idx_df'])[i].sum()[:]

data = pd.merge(data, df2, on = 'idx_df')

del_columns.append('equipment_dict_list')

In [ ]:
data.head(2)

In [ ]:
data = data.sort_values(by = 'idx_df')

# Удаление столбцов

In [ ]:
del_columns

In [ ]:
data.drop((del_columns), axis=1, inplace=True )
data.drop('idx_df', axis=1, inplace=True )


In [ ]:
y = data.query('sample == 1')['price']
X = data.query('sample == 1').drop(['sample'], axis=1)
X = X.drop(['price'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)
X_sub = X_sub.drop(['price'], axis=1)

## Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# **Простые модели**

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [ ]:
def print_report(start, y_test, y_pred):
    print('\nВремя выполнения - ', datetime.now() - start)
    print(f"Точность модели по метрике MAPE:{(mape(y_test, y_pred))*100:0.2f}%")

In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
start = datetime.now()
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
print_report(start, y_test, predict)

# Model 2 : CatBoost

У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

In [ ]:
start = datetime.now()
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2CB_baseline.model')

In [ ]:
# оцениваем точность
'''predict = model.predict(X_test)
print_report(start, y_test, predict)'''

# Время выполнения -  0:02:14.354510
# Точность модели по метрике MAPE:24.04%

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [ ]:
#np.log(y_train)

In [ ]:
start = datetime.now()
cat_boost = CatBoostRegressor(iterations = 4500,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True
                         )
cat_boost.fit(X_train, np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,)

model.save_model('catboost_single_model_3_baseline.model')

In [ ]:
predict_test = np.exp(cat_boost.predict(X_test))
#predict_submission = np.exp(model.predict(X_sub))
print_report(start, y_test, predict_test)

# Время выполнения -  0:02:02.781851
# Точность модели по метрике MAPE:17.00%

# Model 3 : Random Forest

In [ ]:
#start = datetime.now()
# Поиск наилучших параметров. Закомментирован для экономии времени
'''random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 400, num = 4)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(5, 15, num = 6)] + [None],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

rfr = RandomForestRegressor(bootstrap=False, max_features='sqrt',
                      min_samples_split=10, n_estimators=300, random_state=42)
                      rf_random = RandomizedSearchCV(estimator = rfr, param_distributions = random_grid,
                      n_iter = 100, cv = 3, verbose=10, random_state=RANDOM_SEED, n_jobs = -1)
rf_random.fit(X_train, np.log(y_train))
rf_random.best_params_'''

In [ ]:
#rf_random.best_params_

In [ ]:
#best_rfr = rf_random.best_estimator_

In [ ]:
'''best_rfr = RandomForestRegressor(random_state=RANDOM_SEED
                      , n_estimators=200
                      , min_samples_split=2
                      , min_samples_leaf=2
                      , max_features='auto'
                      , max_depth=None
                      , bootstrap=False
                      , n_jobs=-1)

best_rfr.fit(X_train, np.log(y_train))


predict_rfr = np.exp(best_rfr.predict(X_test))
print_report(start, y_test, predict_rfr)'''

#Время выполнения -  0:17:37.642517
#Точность модели по метрике MAPE:25.96%

In [ ]:
#model = best_rfr

In [ ]:
#predict_test = np.exp(model.predict(X_test))
#predict_submission = np.exp(model.predict(X_sub))

In [ ]:
type(sample_submission['price'][8])

# Model 4: Gradient Boosting

In [ ]:
start = datetime.now()

'''random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 400, num = 8)],
               'max_features': ['auto', 'sqrt', 'log2'],
               'max_depth': [int(x) for x in np.linspace(5, 15, num = 6)] + [None],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4]}'''

In [ ]:
# Поиск наилучших параметров. Закомментирован для экономии времени
'''gbr = GradientBoostingRegressor()
gbr_random = RandomizedSearchCV(estimator = gbr, param_distributions = random_grid, n_iter = 100,
                                cv = 3, verbose=10, random_state=RANDOM_SEED, n_jobs = -1)
gbr_random.fit(X_train, np.log(y_train))
gbr_random.best_params_'''

In [ ]:
'''best_gbr = GradientBoostingRegressor(random_state=RANDOM_SEED
                      , n_estimators=314
                      , min_samples_split=2
                      , min_samples_leaf=2
                      , max_features='auto'
                      , max_depth=5)'''

In [ ]:
'''best_gbr.fit(X_train, np.log(y_train))

predict_gbr = np.exp(best_gbr.predict(X_test))
print_report(start, y_test, predict_gbr)

# Время выполнения -  0:12:52.015573
# Точность модели по метрике MAPE:19.08%'''

# Model 5: XGB

In [ ]:
'''start = datetime.now()

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                          learning_rate=0.05, max_depth=12, alpha=1, n_jobs=-1,
                          n_estimators=1000)
xgb_reg.fit(X_train, np.log(y_train))
xgb_red_pred = np.exp(xgb_reg.predict(X_test))
print_report(start, y_test, xgb_red_pred)

#Время выполнения -  1:00:55.337852
#Точность модели по метрике MAPE:16.24%'''

# Выводы
Всего были опробованы следующие модели:

- Linear Regression
- Random Forest Regressor
- Decision Tree Regressor
- AdaBoost
- XGBoost
- CatBoost
- GradientBoosting

Лучшие результаты получены с CatBoost, XGBoost, несколько хуже с GradientBoosting

# **Ансамбли**

# Model 6: Bagging

In [ ]:
# Bagging с Random Forest регрессором
'''start = datetime.now()

bagg_rfr = BaggingRegressor(best_rfr, n_estimators=3, n_jobs=-1, random_state=RANDOM_SEED)
bagg_rfr.fit(X_train, np.log(y_train))
predict_bagg_rfr = np.exp(bagg_rfr.predict(X_test))
print_report(start, y_test, predict_bagg_rfr)'''

# Время выполнения -  0:35:26.661721
# Точность модели по метрике MAPE:21.69%

In [ ]:
# Bagging с Gradient Boosting Regressor
'''start = datetime.now()

bagg_gbr = BaggingRegressor(best_gbr, n_estimators=3, n_jobs=-1, random_state=RANDOM_SEED)
bagg_gbr.fit(X_train, np.log(y_train))
predict_bagg_gbr = np.exp(bagg_gbr.predict(X_test))
print_report(start, y_test, predict_bagg_gbr)'''

# Время выполнения -  0:21:17.588537
# Точность модели по метрике MAPE:19.59%
# Время выполнения -  0:10:35.655910
# Точность модели по метрике MAPE:18.97%

In [ ]:
# Bagging с XGB регрессором
'''start = datetime.now()

bagg_xgb = BaggingRegressor(xgb_reg, n_estimators=3, n_jobs=-1, random_state=RANDOM_SEED)
bagg_xgb.fit(X_train, np.log(y_train))
predict_bagg_xgb = np.exp(bagg_xgb.predict(X_test))
print_report(start, y_test, predict_bagg_xgb)'''

In [ ]:
len(X_test)

# Model 7: Stacking

In [ ]:
'''start = datetime.now()
estimators = [
    ('cat_boost', CatBoostRegressor(random_seed=RANDOM_SEED,
                                    eval_metric='MAPE',
                                    iterations=4500,
                                    l2_leaf_reg=4,
                                    depth=7,
                                    custom_metric=['MAE'],
                                    loss_function='RMSE',
                                    learning_rate=0.08,
                                    verbose=1)
    )
    ('b_gbr', BaggingRegressor(GradientBoostingRegressor(random_state=RANDOM_SEED
                                                                ,n_estimators=314
                                                                ,min_samples_split=2
                                                                ,min_samples_leaf=2
                                                                ,max_features='auto'
                                                                ,max_depth=5)
                                                                ,n_estimators=3
                                                                ,n_jobs=-1
                                                                ,random_state=RANDOM_SEED))
            ,('xgb', xgb.XGBRegressor(objective='reg:squarederror'
                                      ,colsample_bytree=0.5
                                      ,learning_rate=0.03
                                      ,max_depth=12
                                      ,alpha=1
                                      ,n_estimators=1000))
]

stack_ensemble = StackingRegressor(
    estimators=estimators,
    verbose=10,
    passthrough=True,
    n_jobs=-1,
    final_estimator=CatBoostRegressor(random_seed=RANDOM_SEED,
                                    eval_metric='MAPE',
                                    iterations=4500,
                                    l2_leaf_reg=4,
                                    depth=7,
                                    custom_metric=['MAE'],
                                    loss_function='RMSE',
                                    learning_rate=0.08,
                                    verbose=1)
)

stack_ensemble.fit(X_train, np.log(y_train))
y_pred = np.exp(stack_ensemble.predict(X_test))

print_report(start, y_test, y_pred)'''

# Время выполнения -  6:39:02.674098
# Точность модели по метрике MAPE:16.86%

In [ ]:
'''start = datetime.now()

estimators=[('b_gbr', BaggingRegressor(GradientBoostingRegressor(random_state=RANDOM_SEED
                                                                ,n_estimators=314
                                                                ,min_samples_split=2
                                                                ,min_samples_leaf=2
                                                                ,max_features='auto'
                                                                ,max_depth=5)
                                                                ,n_estimators=3
                                                                ,n_jobs=-1
                                                                ,random_state=RANDOM_SEED))
            ,('xgb', xgb.XGBRegressor(objective='reg:squarederror'
                                      ,colsample_bytree=0.5
                                      ,learning_rate=0.03
                                      ,max_depth=11
                                      ,alpha=1
                                      ,n_estimators=1200))]

st_ensemble = StackingRegressor(estimators=estimators
                                ,final_estimator = CatBoostRegressor(iterations = 4500
                                                                     ,random_seed = RANDOM_SEED
                                                                     ,eval_metric='MAPE'
                                                                     ,custom_metric=['MAE']
                                                                     ,silent=True
                                                                     ,learning_rate=0.11
                                                                     ,depth=11
                                                                     ,l2_leaf_reg=7
                                                                     ,random_strength=0.2))

    
st_ensemble.fit(X_train, np.log(y_train))

predict_submission = np.exp(st_ensemble.predict(X_test))
print_report(start, y_test, predict_submission)'''

# Время выполнения -  6:20:06.560627
# Точность модели по метрике MAPE:17.11%

# Время выполнения -  6:21:37.670115
# Точность модели по метрике MAPE:17.10%

In [ ]:
start = datetime.now()
estimators = [
    ('cat_boost', CatBoostRegressor(random_seed=RANDOM_SEED,
                                    eval_metric='MAPE',
                                    iterations=4500,
                                    l2_leaf_reg=4,
                                    depth=7,
                                    custom_metric=['MAE'],
                                    loss_function='RMSE',
                                    learning_rate=0.08,
                                    verbose=1)
    )

]

stack_ensemble = StackingRegressor(
    estimators=estimators,
    verbose=10,
    passthrough=True,
    n_jobs=-1,
    final_estimator= xgb.XGBRegressor(objective='reg:squarederror',
                                     colsample_bytree=0.5,
                                     learning_rate=0.03,
                                     max_depth=11,
                                     alpha=1,
                                     n_jobs=-1,
                                     n_estimators=1100,
                                     seed=RANDOM_SEED)
)

stack_ensemble.fit(X_train, np.log(y_train))
y_pred = np.exp(stack_ensemble.predict(X_test))

print_report(start, y_test, y_pred)

# Время выполнения -  1:17:36.156442
# Точность модели по метрике MAPE:16.61%

# Время выполнения -  1:11:50.108188
# Точность модели по метрике MAPE:15.99%

# Submission

In [ ]:
X_sub.head(3)

In [ ]:
#X_sub = scaler.transform(X_sub)
predict_submission = np.exp(stack_ensemble.predict(X_sub))*0.8795125

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('sample_submission.csv', index=False)
sample_submission.head(10)

# Итоги и содержание работы
- Проведен парсинг свежих данных с auto.ru
- Произведен разведывательный анализ данных
- Были созданы новые признаки
- Была попытка учесть разницу в ценах на момент создания тестовой выборки.
- Были подготовлены данные для использования в моделях машинного обучения
- Проведены тестовые обучения на различных моделях
- Подобраны оптимальные гиперпараметры для моделей
- Был произведен экспериментальный подбор моделей для ансамблей Бэггинга и Стэкинга
- Результаты загружены на Kaggle и GitHub


# Выводы
- наилучшие результаты на наших данных показали модели CatBoost, XGBoost и ансамбли с их использованием. При этом Bagging с однородными моделями не показал значительных улучшений предсказания.

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing